# 3. User profile Training the GNN model

#  Import Libraries 

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# 1. Import encoded dataset 

In [4]:
df = pd.read_csv('haircare_encoded_final2.csv')
df2 = pd.read_csv('hair Care Survey_v2 (Responses) - Form responses 1.csv')

df.head()

,consumed_waterday_l,current_hair_condition,age,hair_porosity,hair_texture,hair_density,harline_condition,hair_breakage,hair_loss_state,hair_length_current_hair_length,...,use_embed_3574,use_embed_3575,use_embed_3576,use_embed_3577,use_embed_3578,use_embed_3579,use_embed_3580,use_embed_3581,use_embed_3582,use_embed_3583
0,NaN,1,1,2,3,2,6,2,0,3,...,0.015701,-0.097407,0.064168,0.018853,0.016023,0.021620,-0.030467,-0.082919,0.027047,0.022251
1,-0.195136,1,2,3,3,3,1,2,1,5,...,0.019354,-0.105700,0.083065,-0.033938,-0.005165,-0.047516,-0.006168,0.029866,0.032657,-0.024623
2,-0.195136,1,1,3,3,3,1,7,1,5,...,0.062959,-0.082641,0.057330,-0.006855,-0.050399,-0.035920,0.010174,0.005829,0.022427,-0.059028
3,0.216102,6,1,1,1,1,0,3,1,6,...,-0.052985,-0.088383,0.016733,0.005497,-0.050175,0.006881,0.041893,0.021341,-0.047707,-0.025795
4,-1.017613,6,1,3,1,2,1,2,2,3,...,0.074204,-0.096372,0.018246,0.040669,-0.018792,0.005182,-0.016291,0.051236,0.019341,-0.020074


# 2. Data cleaning and preparation

In [7]:

# === Check for NaNs ===
if df.isnull().values.any():
    print("⚠️ Found missing values — filling with 0")
    df = df.fillna(0)

# === Step 3: Features and Target ===
X = df.drop(columns=['current_hair_condition'])


# === Label encode string targets like "Dry", "Healthy", etc. ===
label_enc = LabelEncoder()
dfx= df2

dfx['Current Hair condition'] = label_enc.fit_transform(dfx['Current Hair condition'])

if dfx.isnull().values.any():
    dfx = dfx.fillna(0)
    
# === Then do one-hot encoding ===
y = dfx['Current Hair condition'].values
y_cat = to_categorical(y)

# === Save class names for decoding later ===
class_names = label_enc.classes_

⚠️ Found missing values — filling with 0


# 3. Prepare data for training

In [35]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# === Create Graph Structure ===
def create_graph_data(X, y):
    x = torch.tensor(X, dtype=torch.float)
    y = torch.tensor(y, dtype=torch.long)

    # Fully connect all nodes (this is basic; better graphs can use k-NN or similarity)
    edge_index = torch.combinations(torch.arange(x.size(0)), r=2).T
    edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)  # Make it bidirectional

    return Data(x=x, edge_index=edge_index, y=y)

train_data = create_graph_data(X_train, y_train)
test_data = create_graph_data(X_test, y_test)

# ===  Define GNN Model ===
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

class HaircareGCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(HaircareGCN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x = data.x
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# 4. Training the model

In [37]:
# === Initialize Model, Optimizer, Loss ===
model = HaircareGCN(input_dim=X.shape[1], hidden_dim=32, output_dim=len(class_names))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()

# === Training Loop with Validation Metrics ===
print("Training Model...\n")
for epoch in range(1, 101):
    model.train()
    optimizer.zero_grad()
    out = model(train_data)
    loss = criterion(out, train_data.y)
    loss.backward()
    optimizer.step()

    # === Training Accuracy ===
    pred_train = out.argmax(dim=1)
    train_acc = accuracy_score(train_data.y.cpu(), pred_train.cpu())

    # === Validation ===
    model.eval()
    with torch.no_grad():
        out_val = model(test_data)
        val_loss = criterion(out_val, test_data.y)
        pred_val = out_val.argmax(dim=1)
        val_acc = accuracy_score(test_data.y.cpu(), pred_val.cpu())

    # === Print Epoch Metrics ===
    print(f"Epoch {epoch:3d} | Train Loss: {loss.item():.4f} | Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss.item():.4f} | Val Acc: {val_acc:.4f}")

# === Final Evaluation ===
model.eval()
with torch.no_grad():
    final_preds = model(test_data).argmax(dim=1)

y_true = test_data.y.cpu().numpy()
y_pred = final_preds.cpu().numpy()

# Convert to labels
y_true_labels = [class_names[i] for i in y_true]
y_pred_labels = [class_names[i] for i in y_pred]

# === Classification Report ===
from sklearn.metrics import classification_report
print("\n=== Final Classification Report ===\n")
print(classification_report(y_true_labels, y_pred_labels))

Training Model...

Epoch   1 | Train Loss: 1.7289 | Train Acc: 0.0667 | Val Loss: 2.8750 | Val Acc: 0.2222
Epoch   2 | Train Loss: 1.4537 | Train Acc: 0.4857 | Val Loss: 2.9783 | Val Acc: 0.4074
Epoch   3 | Train Loss: 1.4425 | Train Acc: 0.7333 | Val Loss: 4.4392 | Val Acc: 0.2222
Epoch   4 | Train Loss: 1.2146 | Train Acc: 0.7429 | Val Loss: 4.9518 | Val Acc: 0.2222
Epoch   5 | Train Loss: 0.9231 | Train Acc: 0.8571 | Val Loss: 5.0320 | Val Acc: 0.3704
Epoch   6 | Train Loss: 0.7488 | Train Acc: 0.8476 | Val Loss: 5.1131 | Val Acc: 0.3333
Epoch   7 | Train Loss: 0.6863 | Train Acc: 0.8571 | Val Loss: 5.1246 | Val Acc: 0.2593
Epoch   8 | Train Loss: 0.5422 | Train Acc: 0.8857 | Val Loss: 5.0016 | Val Acc: 0.2593
Epoch   9 | Train Loss: 0.5369 | Train Acc: 0.8952 | Val Loss: 4.7838 | Val Acc: 0.2593
Epoch  10 | Train Loss: 0.4195 | Train Acc: 0.9429 | Val Loss: 4.7363 | Val Acc: 0.2963
Epoch  11 | Train Loss: 0.3147 | Train Acc: 0.9238 | Val Loss: 5.0813 | Val Acc: 0.2593
Epoch  12 | T

# 5. Testing recommender

In [39]:
# === Step 6: Haircare Product Mapping ===
product_recommendations = {
    'dry': ['Deep Conditioner', 'Moisturizing Hair Mask', 'Leave-in Cream'],
    'healthy': ['Mild Shampoo', 'Light Conditioner'],
    'dandruff-prone': ['Anti-Dandruff Shampoo', 'Tea Tree Oil'],
    'moisturize': ['Hydration Mist', 'Leave-In Conditioner'],
    'damaged': ['Protein Treatment', 'Split End Serum'],
    'oily': ['Clarifying Shampoo', 'Scalp Scrub'],
    'dry and brittle': ['Hair Butter', 'Moisture Retention Treatment'],
}

# === Step 7: Sample Recommendations ===
print("\nSample Haircare Recommendations Based on GNN Prediction:\n")
for i in range(min(5, len(y_pred))):
    print(f"Sample #{i+1}")
    print("Predicted Condition:", y_pred_labels[i])
    print("Actual Condition   :", y_true_labels[i])
    print("Suggested Products :", product_recommendations.get(y_pred_labels[i].lower(), ["No suggestion found"]))
    print("=" * 40)


Sample Haircare Recommendations Based on GNN Prediction:

Sample #1
Predicted Condition: Dry
Actual Condition   : Healthy
Suggested Products : ['Deep Conditioner', 'Moisturizing Hair Mask', 'Leave-in Cream']
Sample #2
Predicted Condition: Dry
Actual Condition   : Healthy
Suggested Products : ['Deep Conditioner', 'Moisturizing Hair Mask', 'Leave-in Cream']
Sample #3
Predicted Condition: Damaged
Actual Condition   : Damaged
Suggested Products : ['Protein Treatment', 'Split End Serum']
Sample #4
Predicted Condition: Healthy
Actual Condition   : Dry
Suggested Products : ['Mild Shampoo', 'Light Conditioner']
Sample #5
Predicted Condition: Healthy
Actual Condition   : Healthy
Suggested Products : ['Mild Shampoo', 'Light Conditioner']
